In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=11) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=11) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=11) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=11) # kavi IBG paper-trade

In [24]:
%%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
    
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
    
min_rom = 0.3     # 0.5 would be 50% return

base = 0.05       # Upper or Lower base multiple for prices

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_p.contract)
df_p['qual_contracts'] = qual_contracts
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

#****   PREPARE TO SOW !    ****
#________________________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#... build the high-pop-roc dataframe

# read the dataframe pickles from zdata. Ignore the underscores (underlying)
fs = listdir('./zdata/')
fs = [f for f in fs if (f[-3:] == 'pkl') & (f[0] != '_')]
dfs = pd.concat([pd.read_pickle(r'./zdata/'+f) for f in fs], axis=0).reset_index(drop=True).sort_values('rom', ascending=False)

# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.symbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter puts whose underlying price is above the mean.
df3 = df2[(df2.undPrice > df2.avg)]

Wall time: 11.1 s


In [25]:
df3

,symbol,strike,dte,right,undPrice,pop,rom,expPrice,hv,bsmDelta,...,ibprice,close,bid,ask,lot,margin,hi,lo,avg,opt_ticker
183,ADANIPORT,335.0,52,P,378.1,0.866557,0.483742,6.65,0.327875,0.866557,...,NaN,6.00,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=343966221, symbol..."
217,ADANIPORT,330.0,80,P,378.1,0.848625,0.413727,8.75,0.335355,0.848625,...,NaN,7.90,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=347835047, symbol..."
184,ADANIPORT,330.0,52,P,378.1,0.890984,0.403725,5.55,0.327875,0.890984,...,NaN,5.00,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=343966215, symbol..."
595,BANKNIFTY,24900.0,3,P,26960.0,1.000000,0.358733,11.60,0.107522,1.000000,...,NaN,3.05,-1.0,-1.0,20,54324.47,28387.0,23605.5,26139.770747,"Ticker(contract=Option(conId=340762158, symbol..."
137,ADANIPORT,335.0,24,P,378.1,0.957565,0.354623,2.25,0.289243,0.957565,...,NaN,2.00,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=339362217, symbol..."
719,BANKNIFTY,25300.0,80,P,26960.0,0.857088,0.345300,297.75,0.162095,0.857088,...,NaN,263.20,-1.0,-1.0,20,54324.47,28387.0,23605.5,26139.770747,"Ticker(contract=Option(conId=347834231, symbol..."
185,ADANIPORT,325.0,52,P,378.1,0.912314,0.334619,4.60,0.327875,0.912314,...,NaN,4.10,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=343966206, symbol..."
594,BANKNIFTY,25000.0,3,P,26960.0,1.000000,0.327808,10.60,0.107522,1.000000,...,NaN,2.15,-1.0,-1.0,20,54324.47,28387.0,23605.5,26139.770747,"Ticker(contract=Option(conId=340762165, symbol..."
604,BANKNIFTY,24000.0,3,P,26960.0,1.000000,0.316984,10.25,0.107522,1.000000,...,NaN,1.80,-1.0,-1.0,20,54324.47,28387.0,23605.5,26139.770747,"Ticker(contract=Option(conId=340762084, symbol..."
218,ADANIPORT,320.0,80,P,378.1,0.890004,0.304976,6.45,0.335355,0.890004,...,NaN,5.80,-1.0,-1.0,2500,166550.00,452.2,294.1,376.185270,"Ticker(contract=Option(conId=347835038, symbol..."


In [ ]:
# WARNING!!!... Make df the dataframe that you want to execute on!
df = df3.copy()   # make this the last dataframe to get the orders placed

contracts = [c.contract for c in df.ticker]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts consuming {:,} margin from full available funds of {:,}'.format(len(contracts), sum(df.margin), av_funds))

In [ ]:
df_analyze = df[df.ibSymbol.isin(df2.ibSymbol.unique())].groupby(['expiry', 'ibSymbol', 'strike']).apply(max)
df_analyze

In [ ]:
df_analyze.to_excel('./zdata/analyze.xlsx', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'NSE') for s in df2.ibSymbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)

In [ ]:
df_analyze.ibSymbol.unique()

In [ ]:
# Write closed orders placed to file
x  =  r'./zdata/' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df_p.to_excel(writer, 'all-options', index=False, header=1)
writer.save()

In [26]:
df_p

,contract,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,account,ibSymbol,hvstPrice,qual_contracts,harvestOrder
0,"Option(conId=339381880, symbol='AJANTPHAR', la...",-500.0,0.750000,-375.00,4.005505,1627.75,0.0,U9329809,AJANTPHAR,0.45,"Option(conId=339381880, symbol='AJANTPHAR', la...","LimitOrder(action='BUY', totalQuantity=500.0, ..."
1,"Option(conId=339344049, symbol='APOLLOTYR', la...",-3000.0,0.265987,-797.96,1.042168,2328.54,0.0,U9329809,APOLLOTYR,0.15,"Option(conId=339344049, symbol='APOLLOTYR', la...","LimitOrder(action='BUY', totalQuantity=3000.0,..."
2,"Option(conId=339329747, symbol='BANKINDIA', la...",-6000.0,0.025161,-150.97,0.346278,1926.70,0.0,U9329809,BANKINDIA,0.05,"Option(conId=339329747, symbol='BANKINDIA', la...","LimitOrder(action='BUY', totalQuantity=6000.0,..."
3,"Option(conId=339363008, symbol='BHARATFOR', la...",-1200.0,4.250000,-5100.00,2.130947,-2542.86,0.0,U9329809,BHARATFOR,0.75,"Option(conId=339363008, symbol='BHARATFOR', la...","LimitOrder(action='BUY', totalQuantity=1200.0,..."
4,"Option(conId=339363018, symbol='BHARATFOR', la...",-1200.0,3.888188,-4665.83,3.329615,-670.29,0.0,U9329809,BHARATFOR,1.20,"Option(conId=339363018, symbol='BHARATFOR', la...","LimitOrder(action='BUY', totalQuantity=1200.0,..."
5,"Option(conId=339363025, symbol='BHARATFOR', la...",-1200.0,5.573544,-6688.25,4.728061,-1014.58,0.0,U9329809,BHARATFOR,1.70,"Option(conId=339363025, symbol='BHARATFOR', la...","LimitOrder(action='BUY', totalQuantity=1200.0,..."
6,"Option(conId=339363035, symbol='BHARATFOR', la...",-1200.0,8.045900,-9655.08,5.627062,-2902.61,0.0,U9329809,BHARATFOR,2.00,"Option(conId=339363035, symbol='BHARATFOR', la...","LimitOrder(action='BUY', totalQuantity=1200.0,..."
7,"Option(conId=339345130, symbol='BHARTIART', la...",-1700.0,0.724811,-1232.18,1.936071,2059.14,0.0,U9329809,BHARTIART,0.45,"Option(conId=339345130, symbol='BHARTIART', la...","LimitOrder(action='BUY', totalQuantity=1700.0,..."
8,"Option(conId=339336843, symbol='CADILAHC', las...",-1600.0,0.747905,-1196.65,1.585724,1340.51,0.0,U9329809,CADILAHC,0.45,"Option(conId=339336843, symbol='CADILAHC', las...","LimitOrder(action='BUY', totalQuantity=1600.0,..."
9,"Option(conId=339336848, symbol='CADILAHC', las...",-1600.0,1.477397,-2363.83,2.085169,972.44,0.0,U9329809,CADILAHC,0.75,"Option(conId=339336848, symbol='CADILAHC', las...","LimitOrder(action='BUY', totalQuantity=1600.0,..."
